## Import Kaggle datasets into Google Drive

Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Import necessary packages

In [0]:
import os
import zipfile
import time

Run the following code cell to change current working directory

In [0]:
%cd drive
%cd 'My Drive'
%cd CHAMPS_dataset

Use Kaggle API to download CHAMPS dataset - **Only run if dataset isn't already established in your Drive**

In [0]:
os.environ['KAGGLE_USERNAME'] = "abasu89"
os.environ['KAGGLE_KEY'] = "043fae37443c24fe3c56f2018a03e69f"

In [0]:
!kaggle competitions download -c champs-scalar-coupling

Run the following code block for all zipped data files (resulting from the download performed in the above code cell.

In [0]:
zip_ref = zipfile.ZipFile('train.csv.zip', 'r')
zip_ref.extractall()
zip_ref.close()

## Data Wrangling

### Assign datasets to variables

This section should **always** be executed.

Import necessary packages and print files present in current directory.

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import os
import seaborn as sns
import pickle
%matplotlib inline

print(os.listdir())

In [0]:
train_ds = pd.read_csv("train.csv")
scalar_coupling_ctbs = pd.read_csv("scalar_coupling_contributions.csv")
dipole_moments = pd.read_csv("dipole_moments.csv")
magnetic_shld_ts = pd.read_csv("magnetic_shielding_tensors.csv")
mulliken_charge = pd.read_csv("mulliken_charges.csv")
potential_energy = pd.read_csv("potential_energy.csv")
structures = pd.read_csv("structures.csv")

In [0]:
test_ds = pd.read_csv("test.csv")

### Combine datasets

Code in this section only needs to be executed if the Data Visualization section in this notebook is being used.

In [0]:
print('concatenating scalar coupling contributions with train_ds...')
combined_data = pd.concat([train_ds, scalar_coupling_ctbs[['fc','sd','pso','dso']]], axis=1) # merge these 4 columns from scalar_coupling_ctbs into train_ds

In [0]:
print('concatenating structures with train_ds...')
x = combined_data.merge(structures, right_on=['molecule_name', 'atom_index'], left_on=['molecule_name', 'atom_index_1'])
combined_data = x.merge(structures, right_on=['molecule_name', 'atom_index'], left_on=['molecule_name', 'atom_index_0'], suffixes=('_atom_1','_atom_0'))
combined_data.drop(['atom_index_atom_0', 'atom_index_atom_1'], axis=1, inplace=True) # remove redundant columns
arrange_cols = ['id', 'molecule_name', 'type', 'atom_index_0', 'atom_atom_0', 'x_atom_0', 'y_atom_0', 'z_atom_0', 'atom_index_1', 'atom_atom_1', 'x_atom_1', 'y_atom_1', 'z_atom_1', 'fc',
                'sd','pso','dso','scalar_coupling_constant']
combined_data = combined_data[arrange_cols]

In [0]:
print('merging structures with test_ds...')
y = test_ds.merge(structures, right_on=['molecule_name', 'atom_index'], left_on=['molecule_name', 'atom_index_1'])
test_dataset = y.merge(structures, right_on=['molecule_name', 'atom_index'], left_on=['molecule_name', 'atom_index_0'], suffixes=('_atom_1', '_atom_0'))
test_dataset.drop(['atom_index_atom_0','atom_index_atom_1'], axis=1, inplace=True)
test_arrange_cols = ['id', 'molecule_name', 'type','atom_index_0', 'atom_atom_0', 'x_atom_0', 'y_atom_0', 'z_atom_0', 'atom_index_1', 'atom_atom_1', 'x_atom_1', 'y_atom_1', 'z_atom_1']
test_dataset = test_dataset[test_arrange_cols]

In [0]:
print('merging magnetic_shield_tensors with combined_data...')
x = combined_data.merge(magnetic_shld_ts, right_on=['molecule_name', 'atom_index'], left_on=['molecule_name', 'atom_index_1'])
combined_data = x.merge(magnetic_shld_ts, right_on=['molecule_name', 'atom_index'], left_on=['molecule_name', 'atom_index_0'], suffixes=('_atom_1', '_atom_0'))
combined_data.drop(['atom_index_atom_0',  'atom_index_atom_1'], axis=1, inplace=True)
arrange_cols = ['id', 'molecule_name', 'type','atom_index_0', 'atom_atom_0', 'x_atom_0', 'y_atom_0', 'z_atom_0',  'XX_atom_0', 'YX_atom_0', 'ZX_atom_0', 'XY_atom_0', 'YY_atom_0', 
                'ZY_atom_0', 'XZ_atom_0', 'YZ_atom_0', 'ZZ_atom_0','atom_index_1', 'atom_atom_1', 'x_atom_1', 'y_atom_1', 'z_atom_1', 'XX_atom_1', 'YX_atom_1', 'ZX_atom_1', 'XY_atom_1', 
                'YY_atom_1', 'ZY_atom_1', 'XZ_atom_1', 'YZ_atom_1', 'ZZ_atom_1','fc', 'sd', 'pso', 'dso', 'scalar_coupling_constant']
combined_data = combined_data[arrange_cols]

In [0]:
print('merging mulliken_charge with combined_data...')
x = combined_data.merge(mulliken_charge, right_on=['molecule_name', 'atom_index'], left_on=['molecule_name', 'atom_index_1'])
combined_data = x.merge(mulliken_charge, right_on=['molecule_name', 'atom_index'], left_on=['molecule_name', 'atom_index_0'], suffixes=('_atom_1', '_atom_0'))
combined_data.drop(['atom_index_atom_0',  'atom_index_atom_1'], axis=1, inplace=True)
arrange_cols = ['id', 'molecule_name', 'type', 'atom_index_0', 'atom_atom_0', 'x_atom_0', 'y_atom_0', 'z_atom_0',  'XX_atom_0', 'YX_atom_0', 'ZX_atom_0', 'XY_atom_0', 'YY_atom_0', 
                'ZY_atom_0', 'XZ_atom_0', 'YZ_atom_0', 'ZZ_atom_0','mulliken_charge_atom_0','atom_index_1', 'atom_atom_1', 'x_atom_1', 'y_atom_1', 'z_atom_1', 'XX_atom_1', 'YX_atom_1', 
                'ZX_atom_1', 'XY_atom_1', 'YY_atom_1', 'ZY_atom_1', 'XZ_atom_1', 'YZ_atom_1', 'ZZ_atom_1','mulliken_charge_atom_1','fc', 'sd', 'pso', 'dso', 'scalar_coupling_constant']
combined_data = combined_data[arrange_cols]

In [0]:
print('merging dipole moments with combined_data...')
combined_data = combined_data.merge(dipole_moments, right_on='molecule_name', left_on='molecule_name')
arrange_cols = ['id', 'molecule_name', 'type', 'X', 'Y', 'Z','atom_index_0', 'atom_atom_0', 'x_atom_0', 'y_atom_0', 'z_atom_0','XX_atom_0', 'YX_atom_0', 'ZX_atom_0', 'XY_atom_0', 
                'YY_atom_0', 'ZY_atom_0', 'XZ_atom_0', 'YZ_atom_0', 'ZZ_atom_0','mulliken_charge_atom_0','atom_index_1', 'atom_atom_1', 'x_atom_1', 'y_atom_1', 'z_atom_1', 'XX_atom_1', 
                'YX_atom_1', 'ZX_atom_1', 'XY_atom_1', 'YY_atom_1', 'ZY_atom_1', 'XZ_atom_1', 'YZ_atom_1', 'ZZ_atom_1','mulliken_charge_atom_1','fc', 'sd', 'pso', 'dso', 
                'scalar_coupling_constant']
combined_data = combined_data[arrange_cols]

In [0]:
print('merging potential_energy with combined_data...')
combined_data = combined_data.merge(potential_energy, right_on='molecule_name', left_on='molecule_name')
arrange_cols = ['id', 'molecule_name', 'type', 'X', 'Y', 'Z', 'potential_energy','atom_index_0', 'atom_atom_0', 'x_atom_0', 'y_atom_0', 'z_atom_0','XX_atom_0', 'YX_atom_0', 'ZX_atom_0', 
                'XY_atom_0', 'YY_atom_0', 'ZY_atom_0', 'XZ_atom_0', 'YZ_atom_0', 'ZZ_atom_0','mulliken_charge_atom_0','atom_index_1', 'atom_atom_1', 'x_atom_1', 'y_atom_1', 'z_atom_1', 
                'XX_atom_1', 'YX_atom_1', 'ZX_atom_1', 'XY_atom_1', 'YY_atom_1', 'ZY_atom_1', 'XZ_atom_1', 'YZ_atom_1', 'ZZ_atom_1','mulliken_charge_atom_1','fc', 'sd', 'pso', 'dso', 
                'scalar_coupling_constant']
combined_data = combined_data[arrange_cols]

Check for missing data

In [0]:
combined_data.columns[combined_data.isna().any()].tolist()

One-hot encode categorical variables in the training set.

In [0]:
enc = pd.get_dummies(combined_data['type'])
combined_data = combined_data.drop('type', axis=1)
combined_data = combined_data.join(enc)

enc2 = pd.get_dummies(combined_data['atom_atom_0'])
combined_data = combined_data.drop('atom_atom_0', axis=1)
combined_data = combined_data.join(enc2, rsuffix='_atom_0')
combined_data = combined_data.rename(index=str, columns={"H": "H_atom_0"})

enc3 = pd.get_dummies(combined_data['atom_atom_1'])
combined_data = combined_data.drop('atom_atom_1', axis=1)
combined_data = combined_data.join(enc3, rsuffix='_atom_1')
combined_data = combined_data.rename(index=str, columns={"C": 'C_atom_1', "N": "N_atom_1", 'H':'H_atom_1'})

One-hot encode categorical variables in the test set

In [0]:
enc_test = pd.get_dummies(test_dataset['type'])
test_dataset = test_dataset.drop('type', axis=1)
test_dataset = test_dataset.join(enc)

enc2_test = pd.get_dummies(test_dataset['atom_atom_0'])
test_dataset = test_dataset.drop('atom_atom_0', axis=1)
test_dataset = test_dataset.join(enc2, rsuffix='_atom_0')
test_dataset = test_dataset.rename(index=str, columns={"H": "H_atom_0"})

enc3_test = pd.get_dummies(test_dataset['atom_atom_1'])
test_dataset = test_dataset.drop('atom_atom_1', axis=1)
test_dataset = test_dataset.join(enc3, rsuffix='_atom_1')
test_dataset = test_dataset.rename(index=str, columns={"C": 'C_atom_1', "N": "N_atom_1", 'H':'H_atom_1'})

Split training set into dependent and independent variables.

In [0]:
scc_target = combined_data['scalar_coupling_constant'].as_matrix()
combined_data = combined_data.drop('scalar_coupling_constant', axis=1)

Check columns present in training and test sets

In [0]:
combined_data.columns

In [0]:
test_dataset.columns

# **Data Visualizations**

## Exploring the Input Space
The code in this section was adapted from the following Kaggle kernel: https://www.kaggle.com/robikscube/exploring-molecular-properties-data

In [0]:
from sklearn import metrics
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
plt.style.use('ggplot')
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

Distribution of the target variable in the training set

In [0]:
train_ds['scalar_coupling_constant'].plot(kind='hist', figsize=(20, 5), bins=1000, title="Distribution of target variable")
plt.show()

Number of atom pairs in molecules

In [0]:
fig, ax = plt.subplots(1,2)
train_ds.groupby('molecule_name').count().sort_values('id')['id'].plot(kind='hist', bins=25, color=color_pal[6], figsize=(20,5), title='# of Atom Pairs in Molecules in Training Set', ax=ax[0])
test_ds.groupby('molecule_name').count().sort_values('id')['id'].plot(kind='hist', bins=25, color=color_pal[2], figsize=(20,5), title='# of Atom Pairs in Molecules in Test Set', ax=ax[1])
plt.show()

Distribution of Mulliken Charges

In [0]:
mulliken_charge['mulliken_charge'].plot(kind='hist', figsize=(15, 5), bins=500, title='Distribution of Mulliken Charges')
plt.show()

Distribution of Potential Energy

In [0]:
potential_energy['potential_energy'].plot(kind='hist',figsize=(15, 5), bins=500, title='Distribution of Potential Energy', color='b')
plt.show()

Coupling Type Count in Training Set

In [0]:
scalar_coupling_ctbs.groupby('type').count()['molecule_name'].sort_values().plot(kind='barh', color='grey', figsize=(15,5), title='Coupling Type Count in Training Set')
plt.show()

Distribution of Scalar Coupling Contributions

In [0]:
fig, ax = plt.subplots(2, 2, figsize=(20,10))
scalar_coupling_ctbs['fc'].plot(kind='hist', ax=ax.flat[0], bins=500, title='Fermi Contact contribution', color=color_pal[0])
scalar_coupling_ctbs['sd'].plot(kind='hist', ax=ax.flat[1], bins=500, title='Spin-dipolar contribution', color=color_pal[1])
scalar_coupling_ctbs['pso'].plot(kind='hist', ax=ax.flat[2], bins=500, title='Paramagnetic spin-orbit contribution', color=color_pal[2])
scalar_coupling_ctbs['dso'].plot(kind='hist', ax=ax.flat[3], bins=500, title='Diamagnetic spin-orbit contribution', color=color_pal[3])
plt.show()

## Free Form Visualizations

In [0]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
example = structures.loc[structures['molecule_name'] == 'dsgdb9nsd_000011']
ax.scatter(xs=example['x'], ys=example['y'], zs=example['z'], s=100)
plt.suptitle('dsgdb9nsd_000011')
plt.show()

In [0]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
example = structures.loc[structures['molecule_name'] == 'dsgdb9nsd_000010']
ax.scatter(xs=example['x'], ys=example['y'], zs=example['z'], s=100)
plt.suptitle('dsgdb9nsd_000010')
plt.show()

## Visualisations using Matplotlib
This section uses the Matplotlib package to perform visualizations

This code cell plots scalar coupling constant contributions (fc, sd, pso, dso) against the scalar coupling constant (target variable).

In [0]:
fig, ax = plt.subplots(figsize = (10,5))
plt.subplot(2,2,1)
plt.scatter(combined_data['fc'], scc_target)
plt.title('fc vs scalar coupling constant')
plt.subplot(2,2,2)
plt.scatter(combined_data['sd'], scc_target)
plt.title('sd vs scalar coupling constant')
plt.subplot(2,2,3)
plt.scatter(combined_data['pso'], scc_target)
plt.title('pso vs scalar coupling constant')
plt.subplot(2,2,4)
plt.scatter(combined_data['dso'], scc_target)
plt.title('dso vs scalar coupling constant')
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25, wspace=0.35)

This code cell plots fc vs target variable for each coupling type

In [0]:
train = pd.merge(train_ds, scalar_coupling_ctbs, how = 'left',
                  left_on  = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'],
                  right_on = ['molecule_name', 'atom_index_0', 'atom_index_1', 'type'])

fig, ax = plt.subplots(figsize = (20, 10))      
for i, t in enumerate(train['type'].unique()):
    plt.subplot(2, 4, i + 1);
    plt.scatter(train.loc[train['type'] == t, 'fc'], 
                train.loc[train['type'] == t, 'scalar_coupling_constant'], label=t);
    plt.title(f'fc vs target \n for {t} type');

This code cell plots sd vs target variable for each coupling type

In [0]:
fig, ax = plt.subplots(figsize = (20, 10))      
for i, t in enumerate(train['type'].unique()):
    plt.subplot(2, 4, i + 1);
    plt.scatter(train.loc[train['type'] == t, 'sd'], 
                train.loc[train['type'] == t, 'scalar_coupling_constant'], label=t);
    plt.title(f'sd vs target \n for {t} type');

This code cell plots pso vs target variable for each coupling type

In [0]:
fig, ax = plt.subplots(figsize = (20, 10))      
for i, t in enumerate(train['type'].unique()):
    plt.subplot(2, 4, i + 1);
    plt.scatter(train.loc[train['type'] == t, 'pso'], 
                train.loc[train['type'] == t, 'scalar_coupling_constant'], label=t);
    plt.title(f'pso vs target \n for {t} type');

This code cell plots dso vs target variable for each coupling type

In [0]:
fig, ax = plt.subplots(figsize = (20, 10))      
for i, t in enumerate(train['type'].unique()):
    plt.subplot(2, 4, i + 1);
    plt.scatter(train.loc[train['type'] == t, 'dso'], 
                train.loc[train['type'] == t, 'scalar_coupling_constant'], label=t);
    plt.title(f'dso vs target \n for {t} type');

This code cell plots the magnetic shielding tensors vs target variable for Atom 0

In [0]:
fig, ax = plt.subplots(figsize = (20,10))
plt.subplot(3,3,1)
plt.scatter(combined_data['XX_atom_0'], scc_target)
plt.title('XX_atom_0 vs target')
plt.subplot(3,3,2)
plt.scatter(combined_data['YX_atom_0'], scc_target)
plt.title('YX_atom_0 vs target')
plt.subplot(3,3,3)
plt.scatter(combined_data['ZX_atom_0'], scc_target)
plt.title('ZX_atom_0 vs target')
plt.subplot(3,3,4)
plt.scatter(combined_data['XY_atom_0'], scc_target)
plt.title('XY_atom_0 vs target')
plt.subplot(3,3,5)
plt.scatter(combined_data['YY_atom_0'], scc_target)
plt.title('YY_atom_0 vs target')
plt.subplot(3,3,6)
plt.scatter(combined_data['ZY_atom_0'], scc_target)
plt.title('ZY_atom_0 vs target')
plt.subplot(3,3,7)
plt.scatter(combined_data['XZ_atom_0'], scc_target)
plt.title('XZ_atom_0 vs target')
plt.subplot(3,3,8)
plt.scatter(combined_data['YZ_atom_0'], scc_target)
plt.title('YZ_atom_0 vs target')
plt.subplot(3,3,9)
plt.scatter(combined_data['ZZ_atom_0'], scc_target)
plt.title('ZZ_atom_0 vs target')
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25, wspace=0.35)

This code cell plots the magnetic shielding tensors vs target variable for Atom 1

In [0]:
fig, ax = plt.subplots(figsize = (20,10))
plt.subplot(3,3,1)
plt.scatter(combined_data['XX_atom_1'], scc_target)
plt.title('XX_atom_1 vs target')
plt.subplot(3,3,2)
plt.scatter(combined_data['YX_atom_1'], scc_target)
plt.title('YX_atom_1 vs target')
plt.subplot(3,3,3)
plt.scatter(combined_data['ZX_atom_1'], scc_target)
plt.title('ZX_atom_1 vs target')
plt.subplot(3,3,4)
plt.scatter(combined_data['XY_atom_1'], scc_target)
plt.title('XY_atom_1 vs target')
plt.subplot(3,3,5)
plt.scatter(combined_data['YY_atom_1'], scc_target)
plt.title('YY_atom_1 vs target')
plt.subplot(3,3,6)
plt.scatter(combined_data['ZY_atom_1'], scc_target)
plt.title('ZY_atom_1 vs target')
plt.subplot(3,3,7)
plt.scatter(combined_data['XZ_atom_1'], scc_target)
plt.title('XZ_atom_1 vs target')
plt.subplot(3,3,8)
plt.scatter(combined_data['YZ_atom_1'], scc_target)
plt.title('YZ_atom_1 vs target')
plt.subplot(3,3,9)
plt.scatter(combined_data['ZZ_atom_1'], scc_target)
plt.title('ZZ_atom_1 vs target')
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.25, wspace=0.35)

This code cell plots the mulliken charge vs target for both atoms in an atomic pair

In [0]:
fig, ax = plt.subplots(figsize = (20,10))
plt.subplot(1,2,1)
plt.scatter(combined_data['mulliken_charge_atom_0'], scc_target)
plt.title('mulliken_charge_atom_0 vs target')
plt.subplot(1,2,2)
plt.scatter(combined_data['mulliken_charge_atom_1'], scc_target)
plt.title('mulliken_charge_atom_1 vs target')

This code cell plots the dipole moment in each direction vs target variable

In [0]:
fig, ax = plt.subplots(figsize = (20,10))
plt.subplot(1,3,1)
plt.scatter(combined_data['X'], scc_target)
plt.title('dipole moment X vs target')
plt.subplot(1,3,2)
plt.scatter(combined_data['Y'], scc_target)
plt.title('dipole moment Y vs target')
plt.subplot(1,3,3)
plt.scatter(combined_data['Z'], scc_target)
plt.title('dipole moment Z vs target')

This code cell plots the molecular structure coordinates in each axis vs target variable for both atoms in an atomic couple

In [0]:
fig, ax = plt.subplots(figsize = (20,10))
plt.subplot(2,3,1)
plt.scatter(combined_data['x_atom_0'], scc_target)
plt.title('structure x vs target - atom 0')
plt.subplot(2,3,2)
plt.scatter(combined_data['y_atom_0'], scc_target)
plt.title('structure y vs target - atom 0')
plt.subplot(2,3,3)
plt.scatter(combined_data['z_atom_0'], scc_target)
plt.title('structure z vs target - atom 0')
plt.subplot(2,3,4)
plt.scatter(combined_data['x_atom_1'], scc_target)
plt.title('structure x vs target - atom 1')
plt.subplot(2,3,5)
plt.scatter(combined_data['y_atom_1'], scc_target)
plt.title('structure y vs target - atom 1')
plt.subplot(2,3,6)
plt.scatter(combined_data['z_atom_1'], scc_target)
plt.title('structure z vs target - atom 1')

This code cell plots potential energy vs target variable

In [0]:
fig, ax = plt.subplots(figsize = (10,5))
plt.subplot(1,1,1)
plt.scatter(combined_data['potential_energy'], scc_target)
plt.title('potential_energy vs target')

## Visualisations using Seaborn
This section uses the Seaborn package to perform visualizations

This code cell uses a violin plot to visualize the spread of the X, Y, Z coordinates.

In [0]:
sns.set()
d = combined_data[['X', 'Y', 'Z']]
sns.violinplot(data=d, inner="points")

This code cell uses a pair-grid representation to visualize the spread of the X, Y, Z coordinates.

In [0]:
sns.set(style="white")

g = sns.PairGrid(d, diag_sharey=False)
g.map_lower(sns.kdeplot)
g.map_upper(sns.scatterplot)
g.map_diag(sns.kdeplot, lw=3)

# Models

Extract molecular structure data from `combined_data`

In [0]:
trainData_rf = combined_data[['atom_index_0', 'x_atom_0', 'y_atom_0',
       'z_atom_0', 'atom_index_1', 'x_atom_1', 'y_atom_1', 'z_atom_1', '1JHC',
       '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN', 'H_atom_0',
       'C_atom_1', 'H_atom_1', 'N_atom_1']]
testData_rf = test_dataset[['atom_index_0', 'x_atom_0', 'y_atom_0',
       'z_atom_0', 'atom_index_1', 'x_atom_1', 'y_atom_1', 'z_atom_1', '1JHC',
       '1JHN', '2JHC', '2JHH', '2JHN', '3JHC', '3JHH', '3JHN', 'H_atom_0',
       'C_atom_1', 'H_atom_1', 'N_atom_1']]

Split training set into training and validation sets

In [0]:
valid_split = .2 
valid_num = int(np.shape(combined_data)[0] * valid_split)
validSet_rf = trainData_rf.iloc[:valid_num, :]
trainSet_rf = trainData_rf.iloc[valid_num:, :]
valid_scc_rf = scc_target[:valid_num]
train_scc_rf = scc_target[valid_num:]

## [Benchmark] Random Forest using only molecular structure data

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

Train on training set and perform predictions on validation set and evaluate **R^2** score based on validation set predictions

In [0]:
start = time.time()
X, y = trainSet_rf[:50000], train_scc_rf[:50000]
regr = RandomForestRegressor(random_state=0, n_estimators=500, criterion = 'mse')
regr.fit(X, y) 
end = time.time()
print ('The training time is {}'.format(start - end))

In [0]:
bm_score_r2 = regr.score(validSet_rf, valid_scc_rf)
print ('The R^2 score is {}'.format(bm_score_r2))

Compute evaluation metric: **Log of Mean Absolute Error (MAE)** 

In [0]:
y_actual = valid_scc_rf
y_pred = regr.predict(validSet_rf)

In [0]:
eval_score  = np.log(mean_absolute_error(y_actual, y_pred)) / len(y_actual)
print ("The log of MAE is: {}".format(eval_score))

Perform predictions on test set (test.csv), submit to leaderboard to find ranking

In [0]:
np.set_printoptions(suppress='True')

In [0]:
preds = regr.predict(testData_rf)

In [0]:
y_test_pred = np.array(preds).reshape(len(test_ds['id']),1)
ids = (np.array(test_ds['id']).reshape(len(test_ds['id']),1)).astype(int)
submission = np.hstack((ids, y_test_pred))

In [0]:
np.savetxt("predictions.csv", submission, delimiter=",", header="id,scalar_coupling_constant", comments='', fmt="%i,%f")

### Feature Importance

## Neural Network using only molecular structure data

In [0]:
input_size = trainData_rf.shape[1]
hidden_sizes = [128, 64]
output_size = 1

### Simple Neural network using Keras

In [0]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Create and compile neural network.

In [0]:
NN_model = Sequential()
# input layer
NN_model.add(Dense(128, kernel_initializer='normal', input_dim=input_size, activation='relu'))
# hidden layers
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# output layer
NN_model.add(Dense(1, kernel_initializer='normal', activation='linear'))

# compile network
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Define a checkpoint callback:

In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5'
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

Train the model

In [0]:
NN_model.fit(trainSet_rf, train_scc_rf, epochs=8, batch_size=32, validation_split=0.2, callbacks=callbacks_list)

Load weights file of the best model.

In [0]:
weights_file = 'Weights-005--3.21091.hdf5' # choose the best checkpoint 
NN_model.load_weights(weights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

Perform predictions on test set and and save to file for submission to Kaggle.

In [0]:
# perform predictions on test set
preds = NN_model.predict(testData_rf)

In [0]:
np.set_printoptions(suppress='True')
y_test_pred = np.array(preds).reshape(len(test_ds['id']),1)
ids = (np.array(test_ds['id']).reshape(len(test_ds['id']),1)).astype(int)
submission = np.hstack((ids, y_test_pred))

In [0]:
np.savetxt("predictions_NN-8HL.csv", submission, delimiter=",", header="id,scalar_coupling_constant", comments='', fmt="%i,%f")

## Neural Network using Newly Engineered Features

Code in this section has been adapted from the following Kaggle kernel: 
https://www.kaggle.com/todnewman/keras-neural-net-for-champs

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from keras.layers import Dense, Input, Activation
from keras.layers import BatchNormalization,Add,Dropout
from keras.optimizers import Adam
from keras.models import Model, load_model
from keras import callbacks
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings(action="ignore",category=DeprecationWarning)
warnings.filterwarnings(action="ignore",category=FutureWarning)

Reassign variables

In [0]:
df_train = train_ds
df_test = test_ds
df_struct = structures
df_train_sub_charge = mulliken_charge
df_train_sub_tensor = magnetic_shld_ts

This code cell reduces memory usage such that smaller cloud instances, such as Google Colab, can run the kernel.

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
    
print(df_train.shape, df_test.shape, df_struct.shape, df_train_sub_charge.shape, df_train_sub_tensor.shape)
print(df_train.shape, df_test.shape, df_struct.shape, df_train_sub_charge.shape, df_train_sub_tensor.shape)

In [0]:
import psutil
import os

def map_atom_info(df_1,df_2, atom_idx):
    print('Mapping...', df_1.shape, df_2.shape, atom_idx)
    
    df = pd.merge(df_1, df_2.drop_duplicates(subset=['molecule_name', 'atom_index']), how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)

    return df

def show_ram_usage():
    py = psutil.Process(os.getpid())
    print('RAM usage: {} GB'.format(py.memory_info()[0]/2. ** 30))

show_ram_usage()

for atom_idx in [0,1]:
    df_train = map_atom_info(df_train,df_struct, atom_idx)
    df_train = map_atom_info(df_train,df_train_sub_charge, atom_idx)
    df_train = map_atom_info(df_train,df_train_sub_tensor, atom_idx)
    df_train = df_train.rename(columns={'atom': f'atom_{atom_idx}',
                                        'x': f'x_{atom_idx}',
                                        'y': f'y_{atom_idx}',
                                        'z': f'z_{atom_idx}',
                                        'mulliken_charge': f'charge_{atom_idx}',
                                        'XX': f'XX_{atom_idx}',
                                        'YX': f'YX_{atom_idx}',
                                        'ZX': f'ZX_{atom_idx}',
                                        'XY': f'XY_{atom_idx}',
                                        'YY': f'YY_{atom_idx}',
                                        'ZY': f'ZY_{atom_idx}',
                                        'XZ': f'XZ_{atom_idx}',
                                        'YZ': f'YZ_{atom_idx}',
                                        'ZZ': f'ZZ_{atom_idx}',})
    df_test = map_atom_info(df_test,df_struct, atom_idx)
    df_test = df_test.rename(columns={'atom': f'atom_{atom_idx}',
                                'x': f'x_{atom_idx}',
                                'y': f'y_{atom_idx}',
                                'z': f'z_{atom_idx}'})
    #add some features
    df_struct['c_x']=df_struct.groupby('molecule_name')['x'].transform('mean')
    df_struct['c_y']=df_struct.groupby('molecule_name')['y'].transform('mean')
    df_struct['c_z']=df_struct.groupby('molecule_name')['z'].transform('mean')
    df_struct['atom_n']=df_struct.groupby('molecule_name')['atom_index'].transform('max')
    
    show_ram_usage()
    print(df_train.shape, df_test.shape)


In [0]:
def make_features(df):
    df['dx']=df['x_1']-df['x_0']
    df['dy']=df['y_1']-df['y_0']
    df['dz']=df['z_1']-df['z_0']
    df['distance']=(df['dx']**2+df['dy']**2+df['dz']**2)**(1/2)
    return df

df_train=make_features(df_train)
df_test=make_features(df_test) 
test_prediction=np.zeros(len(df_test))
show_ram_usage()
print(df_train.shape, df_test.shape)

def get_dist(df):
    df_temp=df.loc[:,["molecule_name","atom_index_0","atom_index_1","distance","x_0","y_0","z_0","x_1","y_1","z_1"]].copy()
    df_temp_=df_temp.copy()
    df_temp_= df_temp_.rename(columns={'atom_index_0': 'atom_index_1',
                                       'atom_index_1': 'atom_index_0',
                                       'x_0': 'x_1',
                                       'y_0': 'y_1',
                                       'z_0': 'z_1',
                                       'x_1': 'x_0',
                                       'y_1': 'y_0',
                                       'z_1': 'z_0'})
    df_temp_all=pd.concat((df_temp,df_temp_),axis=0)

    df_temp_all["min_distance"]=df_temp_all.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('min')
    df_temp_all["max_distance"]=df_temp_all.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('max')
    
    df_temp= df_temp_all[df_temp_all["min_distance"]==df_temp_all["distance"]].copy()
    df_temp=df_temp.drop(['x_0','y_0','z_0','min_distance'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_closest',
                                         'distance': 'distance_closest',
                                         'x_1': 'x_closest',
                                         'y_1': 'y_closest',
                                         'z_1': 'z_closest'})
    
    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_closest': f'atom_index_closest_{atom_idx}',
                                        'distance_closest': f'distance_closest_{atom_idx}',
                                        'x_closest': f'x_closest_{atom_idx}',
                                        'y_closest': f'y_closest_{atom_idx}',
                                        'z_closest': f'z_closest_{atom_idx}'})
        
    df_temp= df_temp_all[df_temp_all["max_distance"]==df_temp_all["distance"]].copy()
    df_temp=df_temp.drop(['x_0','y_0','z_0','max_distance'], axis=1)
    df_temp= df_temp.rename(columns={'atom_index_0': 'atom_index',
                                         'atom_index_1': 'atom_index_farthest',
                                         'distance': 'distance_farthest',
                                         'x_1': 'x_farthest',
                                         'y_1': 'y_farthest',
                                         'z_1': 'z_farthest'})
        
    for atom_idx in [0,1]:
        df = map_atom_info(df,df_temp, atom_idx)
        df = df.rename(columns={'atom_index_farthest': f'atom_index_farthest_{atom_idx}',
                                        'distance_farthest': f'distance_farthest_{atom_idx}',
                                        'x_farthest': f'x_farthest_{atom_idx}',
                                        'y_farthest': f'y_farthest_{atom_idx}',
                                        'z_farthest': f'z_farthest_{atom_idx}'})
    return df
df_test=(get_dist(df_test))    
df_train=(get_dist(df_train)) 

print(df_train.shape, df_test.shape)
show_ram_usage()


In [0]:
def add_features(df):
    df["distance_center0"]=((df['x_0']-df['c_x'])**2+(df['y_0']-df['c_y'])**2+(df['z_0']-df['c_z'])**2)**(1/2)
    df["distance_center1"]=((df['x_1']-df['c_x'])**2+(df['y_1']-df['c_y'])**2+(df['z_1']-df['c_z'])**2)**(1/2)
    df["distance_c0"]=((df['x_0']-df['x_closest_0'])**2+(df['y_0']-df['y_closest_0'])**2+(df['z_0']-df['z_closest_0'])**2)**(1/2)
    df["distance_c1"]=((df['x_1']-df['x_closest_1'])**2+(df['y_1']-df['y_closest_1'])**2+(df['z_1']-df['z_closest_1'])**2)**(1/2)
    df["distance_f0"]=((df['x_0']-df['x_farthest_0'])**2+(df['y_0']-df['y_farthest_0'])**2+(df['z_0']-df['z_farthest_0'])**2)**(1/2)
    df["distance_f1"]=((df['x_1']-df['x_farthest_1'])**2+(df['y_1']-df['y_farthest_1'])**2+(df['z_1']-df['z_farthest_1'])**2)**(1/2)
    df["vec_center0_x"]=(df['x_0']-df['c_x'])/(df["distance_center0"]+1e-10)
    df["vec_center0_y"]=(df['y_0']-df['c_y'])/(df["distance_center0"]+1e-10)
    df["vec_center0_z"]=(df['z_0']-df['c_z'])/(df["distance_center0"]+1e-10)
    df["vec_center1_x"]=(df['x_1']-df['c_x'])/(df["distance_center1"]+1e-10)
    df["vec_center1_y"]=(df['y_1']-df['c_y'])/(df["distance_center1"]+1e-10)
    df["vec_center1_z"]=(df['z_1']-df['c_z'])/(df["distance_center1"]+1e-10)
    df["vec_c0_x"]=(df['x_0']-df['x_closest_0'])/(df["distance_c0"]+1e-10)
    df["vec_c0_y"]=(df['y_0']-df['y_closest_0'])/(df["distance_c0"]+1e-10)
    df["vec_c0_z"]=(df['z_0']-df['z_closest_0'])/(df["distance_c0"]+1e-10)
    df["vec_c1_x"]=(df['x_1']-df['x_closest_1'])/(df["distance_c1"]+1e-10)
    df["vec_c1_y"]=(df['y_1']-df['y_closest_1'])/(df["distance_c1"]+1e-10)
    df["vec_c1_z"]=(df['z_1']-df['z_closest_1'])/(df["distance_c1"]+1e-10)
    df["vec_f0_x"]=(df['x_0']-df['x_farthest_0'])/(df["distance_f0"]+1e-10)
    df["vec_f0_y"]=(df['y_0']-df['y_farthest_0'])/(df["distance_f0"]+1e-10)
    df["vec_f0_z"]=(df['z_0']-df['z_farthest_0'])/(df["distance_f0"]+1e-10)
    df["vec_f1_x"]=(df['x_1']-df['x_farthest_1'])/(df["distance_f1"]+1e-10)
    df["vec_f1_y"]=(df['y_1']-df['y_farthest_1'])/(df["distance_f1"]+1e-10)
    df["vec_f1_z"]=(df['z_1']-df['z_farthest_1'])/(df["distance_f1"]+1e-10)
    df["vec_x"]=(df['x_1']-df['x_0'])/df["distance"]
    df["vec_y"]=(df['y_1']-df['y_0'])/df["distance"]
    df["vec_z"]=(df['z_1']-df['z_0'])/df["distance"]
    df["cos_c0_c1"]=df["vec_c0_x"]*df["vec_c1_x"]+df["vec_c0_y"]*df["vec_c1_y"]+df["vec_c0_z"]*df["vec_c1_z"]
    df["cos_f0_f1"]=df["vec_f0_x"]*df["vec_f1_x"]+df["vec_f0_y"]*df["vec_f1_y"]+df["vec_f0_z"]*df["vec_f1_z"]
    df["cos_center0_center1"]=df["vec_center0_x"]*df["vec_center1_x"]+df["vec_center0_y"]*df["vec_center1_y"]+df["vec_center0_z"]*df["vec_center1_z"]
    df["cos_c0"]=df["vec_c0_x"]*df["vec_x"]+df["vec_c0_y"]*df["vec_y"]+df["vec_c0_z"]*df["vec_z"]
    df["cos_c1"]=df["vec_c1_x"]*df["vec_x"]+df["vec_c1_y"]*df["vec_y"]+df["vec_c1_z"]*df["vec_z"]
    df["cos_f0"]=df["vec_f0_x"]*df["vec_x"]+df["vec_f0_y"]*df["vec_y"]+df["vec_f0_z"]*df["vec_z"]
    df["cos_f1"]=df["vec_f1_x"]*df["vec_x"]+df["vec_f1_y"]*df["vec_y"]+df["vec_f1_z"]*df["vec_z"]
    df["cos_center0"]=df["vec_center0_x"]*df["vec_x"]+df["vec_center0_y"]*df["vec_y"]+df["vec_center0_z"]*df["vec_z"]
    df["cos_center1"]=df["vec_center1_x"]*df["vec_x"]+df["vec_center1_y"]*df["vec_y"]+df["vec_center1_z"]*df["vec_z"]
    df=df.drop(['vec_c0_x','vec_c0_y','vec_c0_z','vec_c1_x','vec_c1_y','vec_c1_z',
                'vec_f0_x','vec_f0_y','vec_f0_z','vec_f1_x','vec_f1_y','vec_f1_z',
                'vec_center0_x','vec_center0_y','vec_center0_z','vec_center1_x','vec_center1_y','vec_center1_z',
                'vec_x','vec_y','vec_z'], axis=1)
    return df
    
df_train=add_features(df_train)
df_test=add_features(df_test)
print(df_train.shape, df_test.shape)
show_ram_usage()

In [0]:
def create_nn_model(input_shape):
    inp = Input(shape=(input_shape,))
    x = Dense(256)(inp)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    #x = Dropout(0.4)(x)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.4)(x)
    out1 = Dense(2, activation="linear")(x)#mulliken charge 2
    out2 = Dense(6, activation="linear")(x)#tensor 6(xx,yy,zz)
    out3 = Dense(12, activation="linear")(x)#tensor 12(others) 
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    x = Dense(128)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dense(64)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.05)(x)
    x = Dropout(0.2)(x)
    out = Dense(1, activation="linear")(x)#scalar_coupling_constant    
    model = Model(inputs=inp, outputs=[out,out1,out2,out3])
    return model

In [0]:
from datetime import datetime

mol_types=df_train["type"].unique()
cv_score=[]
cv_score_total=0
epoch_n = 400
verbose = 0
batch_size = 2048
    
# Set to True if we want to train from scratch. False will reuse saved models as a starting point.
retrain =True

# Set up GPU preferences
config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 2} ) 
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.6
sess = tf.Session(config=config) 
K.set_session(sess)

start_time=datetime.now()

# Loop through each molecule type
for mol_type in mol_types:
    model_name_rd = ('../keras-neural-net-for-champs/molecule_model_%s.hdf5' % mol_type)
    model_name_wrt = ('/kaggle/working/molecule_model_%s.hdf5' % mol_type)
    print('Training %s' % mol_type, 'out of', mol_types, '\n')
    
    df_train_=df_train[df_train["type"]==mol_type]
    df_test_=df_test[df_test["type"]==mol_type]
    
    # Choose features
    input_features=["x_0","y_0","z_0","x_1","y_1","z_1","c_x","c_y","c_z",
                    'x_closest_0','y_closest_0','z_closest_0','x_closest_1','y_closest_1','z_closest_1',
                    "distance","distance_center0","distance_center1", "distance_c0","distance_c1","distance_f0","distance_f1",
                    "cos_c0_c1","cos_f0_f1","cos_center0_center1","cos_c0","cos_c1","cos_f0","cos_f1","cos_center0","cos_center1",
                    "atom_n"
                   ]
    
    # Standard Scaler from sklearn does seem to work better here than other Scalers
    input_data=StandardScaler().fit_transform(pd.concat([df_train_.loc[:,input_features],df_test_.loc[:,input_features]]))
    
    target_data=df_train_.loc[:,"scalar_coupling_constant"].values
    target_data_1=df_train_.loc[:,["charge_0","charge_1"]]
    target_data_2=df_train_.loc[:,["XX_0","YY_0","ZZ_0","XX_1","YY_1","ZZ_1"]]
    target_data_3=df_train_.loc[:,["YX_0","ZX_0","XY_0","ZY_0","XZ_0","YZ_0","YX_1","ZX_1","XY_1","ZY_1","XZ_1","YZ_1"]]
    
    #following parameters should be adjusted to control the loss function
    #if all parameters are zero, attractors do not work. (-> simple neural network)
    m1=1
    m2=4
    m3=1
    target_data_1=m1*(StandardScaler().fit_transform(target_data_1))
    target_data_2=m2*(StandardScaler().fit_transform(target_data_2))
    target_data_3=m3*(StandardScaler().fit_transform(target_data_3))
    
    # Simple split to provide us a validation set to do our CV checks with
    train_index, cv_index = train_test_split(np.arange(len(df_train_)),random_state=111, test_size=0.1)
    
    # Split all our input and targets by train and cv indexes
    train_input=input_data[train_index]
    cv_input=input_data[cv_index]
    train_target=target_data[train_index]
    cv_target=target_data[cv_index]
    train_target_1=target_data_1[train_index]
    cv_target_1=target_data_1[cv_index]
    train_target_2=target_data_2[train_index]
    cv_target_2=target_data_2[cv_index]
    train_target_3=target_data_3[train_index]
    cv_target_3=target_data_3[cv_index]
    test_input=input_data[len(df_train_):,:]

    # Build the neural net
    nn_model=create_nn_model(train_input.shape[1])
    
    # If retrain==False, then load a previous saved model as a starting point.
    if not retrain:
        nn_model = load_model(model_name_rd)
        
    nn_model.compile(loss='mae', optimizer=Adam())
    
    # Callback for Early Stopping... May want to raise the min_delta for small numbers of epochs
    es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=8,verbose=1, mode='auto', restore_best_weights=True)
    # Callback for Reducing the Learning Rate... when the monitor levels out for 'patience' epochs, then the LR is reduced
    rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=7, min_lr=1e-6, mode='auto', verbose=1)
    # Save the best value of the model for future use
    sv_mod = callbacks.ModelCheckpoint(model_name_wrt, monitor='val_loss', save_best_only=True, period=1)  

    cv_predict=nn_model.predict(cv_input)
    #plot_history(history, mol_type)
    
    accuracy=np.mean(np.abs(cv_target-cv_predict[0][:,0]))
    cv_score.append(np.log(accuracy))
    cv_score_total+=np.log(accuracy)
    
    # Predict on the test data set using our trained model
    test_predict=nn_model.predict(test_input)
    
    # for each molecule type we'll grab the predicted values
    test_prediction[df_test["type"]==mol_type]=test_predict[0][:,0]
    K.clear_session()

cv_score_total/=len(mol_types)

In [0]:
def submit(predictions):
    submit = pd.read_csv('sample_submission.csv')
    print(len(submit), len(predictions))   
    submit["scalar_coupling_constant"] = predictions
    submit.to_csv("predictions_test.csv", index=False)
submit(test_prediction)

print ('Total training time: ', datetime.now() - start_time)

i=0
for mol_type in mol_types: 
    print(mol_type,": cv score is ",cv_score[i])
    i+=1
print("total cv score is",cv_score_total)

### Feature Importance

In [0]:
train_input.shape

In [0]:
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance

model = KerasRegressor(build_fn=nn_model)
model.fit(x=train_input, y=train_target)

perm = PermutationImportance(nn_model, random_state=1).fit(train_input, train_target)
eli5.show_weights(perm, feature_names=input_features)

In [0]:
! pip install eli5